In [1]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from konlpy.tag import Okt  
okt = Okt() 
import tensorflow as tf
import numpy as np
from collections import Counter

from wordcloud import WordCloud
import matplotlib.pyplot as plt


import urllib.request
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
import matplotlib.pyplot as plt 
from string import punctuation
import warnings
warnings.filterwarnings('ignore')

In [4]:
test = pd.read_csv('sort_total_years.csv')
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4562 entries, 0 to 4561
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   song_id     4562 non-null   int64 
 1   song_name   4562 non-null   object
 2   artist      4562 non-null   object
 3   album       4562 non-null   object
 4   Like_Count  4562 non-null   int64 
 5   Lyric       4562 non-null   object
 6   cover_url   4562 non-null   object
 7   tags        4562 non-null   object
 8   year        4562 non-null   object
dtypes: int64(2), object(7)
memory usage: 320.9+ KB


In [5]:
sort_like2000 = test[test['year']=='2000y']
print('2000년대 곡 개수 : ',len(sort_like2000))
sort_like2000.head(2)

2000년대 곡 개수 :  1028


,song_id,song_name,artist,album,Like_Count,Lyric,cover_url,tags,year
1374,418253,보고 싶다,김범수,3rd. 보고 싶다,342238,아무리 기다려도 난 못가 바보처럼 울고 있는 너의 곁에 상처만 주는 나를 왜 ...,https://image.bugsm.co.kr/album/images/200/262...,"['국내', '가요', '노래방', '2000년대', '연대별', 'Y세대', '연...",2000y
1375,445413,사랑합니다...,팀,Tim 영민,332812,나빠요 참 그대란 사람 허락도 없이 왜 내 맘 가져요 그대 때문에 난 힘겹게...,https://image.bugsm.co.kr/album/images/200/307...,"['국내', '가요', '노래방', '2000년대', '연대별', 'Y세대', '연...",2000y


In [6]:
 # 불용어 (가사 빈도수 높은 + 감정분류와 무관한 단어 추가 중)
stop_w = ['all','이렇게','네가','있는','니가','없는','너의','너무','그런',
          'oh','whoo','tuesday','내가','너를','나를','we','this','the','그렇게',
          'so','am','baby','and','can','you','much','me','for','go','in',
          '은', '는', '이', '가', '하','부터','처럼','까지',
          'know','no','of','let','my','수','너','내','나','그','난','봐',
          '돼','건','모든','에서','에게','싶어','잖아',
          '날','널','수','것','못','말','넌','젠','하나','정말','알','여기',
          '우리','다시','하게','니까',
          '때','아','더','게','또','채','일','걸','누구','나는','너는','라면',
          '같아','있어','지금',
          '의','가','보','들','좀','잘','걍','과','도','를','으로','우린','하지',
          '해도','하고','없어','않아',
          '자','에','와','한','하다','네','있다','나의','해','다','내게','왜',
          '거야','이제','그냥','했던','하는']

In [7]:
stop_w = set(stop_w)

## 토큰화

In [8]:
sort_like2000['Lyric'] = sort_like2000['Lyric'].apply(lambda x: [word for word in okt.nouns(x) if word not in stop_w])

In [9]:
# sort_like2000['Lyric'] = sort_like2000['Lyric'].apply(lambda x: [word for word in okt.morphs(x, stem=True) if word not in stop_w])

In [10]:
# sort_like2000['Lyric'] # okt.morphs(x, stem=True)

In [11]:
sort_like2000['Lyric'] # okt.nouns(x)

1374    [바보, 울, 곁, 상처, 보고, 보고, 질, 울, 무릎, 모두, 사랑, 기억, 추...
1375    [대란, 사람, 허락, 맘, 그대, 때문, 눈길, 줄, 다만, 가끔, 미소, 순, ...
1376    [어쩌면, 오늘이, 마지막, 사랑, 마음, 니, 눈, 마주, 치, 사람, 고민, 제...
1377    [술, 잔, 밤, 시절, 모두, 한숨, 그대, 얼굴, 혹시, 지나, 먼저, 그때, ...
1378    [손짓, 목소리, 마음, 사랑, 맘, 가기, 사랑, 맘, 사랑, 사람, 맘, 수도,...
                              ...                        
2397    [안녕, 고하, 바람, 저녁, 강, 길, 위, 나그네, 울, 땅, 거, 방랑, 발원...
2398    [공간, 속, 마음, 미련, 따위, 자존심, 때문, 사랑, 후회, 곧, 한번, 사랑...
2399    [꽃, 곁, 노래, 제비, 언덕, 라보, 귀, 노래, 소리, 꽃, 봄, 알리, 당신...
2400    [오해, 적, 리더, 신문, 주머니, 법전, 손바닥, 욕심, 뱃속, 깊이, 양심, ...
2401    [연분홍, 꽃, 누, 누, 바람결, 들꽃, 바람, 꽃, 향기, 자유, 온, 누리, ...
Name: Lyric, Length: 1028, dtype: object

In [12]:
word2000 = sort_like2000['Lyric']

## corpus 생성
### (1) 단어를 id로 바꾸고 뜻을 dictionary로 만들기

In [13]:
from gensim import corpora

# 각 단어를 (단어id, 나온횟수) 로 바꾸는 작업
dictionary = corpora.Dictionary(word2000)
corpus = [dictionary.doc2bow(text) for text in word2000]

### (2) 코퍼스와 딕셔너리 확인

In [14]:
# corpus[i] : i번째 노래에서 나온단어에서 (단어id, 나온횟수)들을 저장한 list
print(corpus[1]) 

# corpus 사전 단어수
print(len(dictionary))

# dictionary[j] : id값을 j를 가진 단어가 무엇인지 확인
print(dictionary[1111])

[(9, 4), (17, 1), (18, 2), (19, 4), (20, 1), (21, 1), (22, 1), (23, 1), (24, 2), (25, 1), (26, 1), (27, 2), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 2), (38, 1), (39, 1), (40, 2), (41, 1), (42, 1), (43, 2), (44, 1), (45, 2), (46, 2), (47, 1), (48, 1)]
5453
우려


## 3. LDA 모델 훈련
### (1) 훈련

In [15]:
import gensim

# N개의 토픽, k=3
NUM_TOPICS = 3

# passes : 알고리즘 동작횟수, num_words : 각 토픽별 출력할 단어
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.017*"니" + 0.015*"속" + 0.014*"사랑" + 0.013*"맘" + 0.012*"그대"')
(1, '0.080*"사랑" + 0.051*"그대" + 0.018*"사람" + 0.014*"눈물" + 0.014*"가슴"')
(2, '0.017*"사랑" + 0.016*"나나" + 0.013*"속" + 0.010*"안" + 0.009*"끝"')


### (2) 토픽별 10개 표시

In [16]:
# 각 토픽별 10개의 단어를 단어를 출력 (위 코드에서 num_words=10을 한것)
for i in range(3):
    print(ldamodel.print_topics()[i])

(0, '0.017*"니" + 0.015*"속" + 0.014*"사랑" + 0.013*"맘" + 0.012*"그대" + 0.011*"세상" + 0.010*"기억" + 0.010*"눈" + 0.010*"밤" + 0.009*"꿈"')
(1, '0.080*"사랑" + 0.051*"그대" + 0.018*"사람" + 0.014*"눈물" + 0.014*"가슴" + 0.013*"맘" + 0.011*"니" + 0.011*"눈" + 0.008*"마음" + 0.008*"생각"')
(2, '0.017*"사랑" + 0.016*"나나" + 0.013*"속" + 0.010*"안" + 0.009*"끝" + 0.008*"세상" + 0.007*"마음" + 0.007*"그녀" + 0.006*"하늘" + 0.006*"니"')


---

In [17]:
from gensim import corpora
dictionary = corpora.Dictionary(word2000)
corpus = [dictionary.doc2bow(text) for text in word2000]
pd.DataFrame(corpus)

,0,1,2,3,4,5,6,7,8,9,...,207,208,209,210,211,212,213,214,215,216
0,"(0, 1)","(1, 2)","(2, 1)","(3, 1)","(4, 1)","(5, 1)","(6, 1)","(7, 2)","(8, 6)","(9, 4)",...,None,None,None,None,None,None,None,None,None,None
1,"(9, 4)","(17, 1)","(18, 2)","(19, 4)","(20, 1)","(21, 1)","(22, 1)","(23, 1)","(24, 2)","(25, 1)",...,None,None,None,None,None,None,None,None,None,None
2,"(1, 3)","(4, 1)","(9, 13)","(22, 1)","(24, 3)","(35, 6)","(46, 1)","(49, 1)","(50, 3)","(51, 1)",...,None,None,None,None,None,None,None,None,None,None
3,"(4, 1)","(9, 2)","(11, 1)","(19, 6)","(24, 1)","(35, 3)","(55, 1)","(78, 2)","(79, 4)","(80, 2)",...,None,None,None,None,None,None,None,None,None,None
4,"(0, 1)","(9, 11)","(12, 1)","(18, 1)","(31, 8)","(35, 3)","(37, 1)","(41, 1)","(55, 4)","(100, 1)",...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1023,"(11, 1)","(62, 1)","(66, 1)","(133, 1)","(155, 1)","(156, 1)","(218, 1)","(299, 1)","(336, 4)","(378, 2)",...,None,None,None,None,None,None,None,None,None,None
1024,"(1, 1)","(4, 2)","(9, 7)","(16, 1)","(29, 1)","(37, 1)","(42, 3)","(46, 1)","(55, 1)","(60, 1)",...,None,None,None,None,None,None,None,None,None,None
1025,"(0, 1)","(36, 1)","(40, 1)","(55, 1)","(119, 1)","(200, 1)","(298, 2)","(384, 1)","(386, 1)","(527, 1)",...,None,None,None,None,None,None,None,None,None,None
1026,"(35, 1)","(111, 3)","(151, 1)","(294, 1)","(425, 1)","(449, 1)","(459, 1)","(488, 1)","(548, 1)","(965, 1)",...,None,None,None,None,None,None,None,None,None,None


In [18]:
from tqdm import tqdm 
import re 
from gensim.models.ldamodel import LdaModel 
from gensim.models.callbacks import CoherenceMetric 
from gensim import corpora 
from gensim.models.callbacks import PerplexityMetric
from gensim.models.coherencemodel import CoherenceModel

In [ ]:
# Compute Coherence Score using c_v 
coherence_model_lda = CoherenceModel(model=ldamodel, texts=word2000, dictionary=dictionary, coherence='c_v') 
coherence_lda = coherence_model_lda.get_coherence() 
print('\nCoherence Score (c_v): ', coherence_lda)  # c_v는 0과 1사이로 0.55 정도면 준수
# Compute Coherence Score using UMass 
coherence_model_lda = CoherenceModel(model=ldamodel, texts=word2000, dictionary=dictionary, coherence="u_mass") 
coherence_lda = coherence_model_lda.get_coherence() 
print('\nCoherence Score (u_mass): ', coherence_lda) # u_mass는 0에 가까울수록 완벽한 일관성을 가진다는 의미


In [19]:
sort_like2000.head(1)

,song_id,song_name,artist,album,Like_Count,Lyric,cover_url,tags,year
1374,418253,보고 싶다,김범수,3rd. 보고 싶다,342238,"[바보, 울, 곁, 상처, 보고, 보고, 질, 울, 무릎, 모두, 사랑, 기억, 추...",https://image.bugsm.co.kr/album/images/200/262...,"['국내', '가요', '노래방', '2000년대', '연대별', 'Y세대', '연...",2000y


## Error
AttributeError: 'list' object has no attribute 'lower' gensim
> https://stackoverflow.com/questions/41829323/attributeerror-list-object-has-no-attribute-lower-gensim<br>
> df.Lyric.astype(str) 수정

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# 설정해준 카테고리의 데이터들만 추출
year_df = sort_like2000
# CountVectorizer로 텍스트 데이터들 단어 빈도수에 기반해 벡터화시키기(fit_transform까지!)
count_vect = CountVectorizer(max_df=0.95, max_features=1000,
                            min_df=2, stop_words=stop_w,
                            ngram_range=(1,2))
ftr_vect = count_vect.fit_transform(year_df.Lyric.astype(str))
# LDA클래스를 이용해서 피처 벡터화시킨 것을 토픽모델링 시키기
# 8개의 주제만 뽑았으니 n_components(토픽개수) 8로 설정
lda = LatentDirichletAllocation(n_components=3, random_state=42)
lda.fit(ftr_vect)
# components_속성은 8개의 토픽별(row)로 1000개의 feature(단어)들의 분포수치(column)를 보여줌
print(lda.components_.shape)
print(lda.components_)

(3, 1000)
[[ 0.34169767 13.97581759 40.24322545 ...  0.34798934  0.33617215
   0.33643299]
 [18.57242901 17.32934242 52.18287776 ... 18.9139354  13.32116455
  48.32870749]
 [ 6.08587332 15.69483999 25.57389679 ...  0.73807526  0.3426633
   0.33485952]]


In [21]:
# 이 때 lda_model이란, 벡터화시킨 텍스트 데이터를 fit까지만 적용한 모델!
def display_topic_words(lda_model, feature_names, num_top_words):
    for topic_idx, topic in enumerate(lda_model.components_):
        print('\nTopic #', topic_idx+1)
        
        # Topic별로 1000개의 단어들(features)중에서 높은 값 순으로 정렬 후 index를 반환해줌!
        # argsort()는 디폴트가 오름차순임(1,2,3,...) 그래서 [::-1]로 내림차순으로 바꿔주기
        topic_word_idx = topic.argsort()[::-1]
        top_idx = topic_word_idx[:num_top_words]
        
        # CountVectorizer함수 할당시킨 객체에 get_feature_names()로 벡터화시킨 feature(단어들)볼 수 있음!
        # 이 벡터화시킨 단어들(features)은 숫자-알파벳순으로 정렬되며, 단어들 순서는 fit_transform시키고 난 이후에도 동일!
        # '문자열'.join 함수로 특정 문자열 사이에 끼고 문자열 합쳐줄 수 있음.
        feature_concat = '+'.join([str(feature_names[i])+'*'+str(round(topic[i], 1)) for i in top_idx])
        print(feature_concat)        
feature_names = count_vect.get_feature_names()
display_topic_words(lda, feature_names, 15)


Topic # 1
그대*1751.9+사랑*1129.0+사람*392.7+눈물*306.9+기억*235.9+그대 사랑*207.3+사랑 그대*205.8+그대 그대*170.3+생각*164.9+마음*148.2+모두*148.0+시간*147.2+가슴*137.4+바보*135.0+혼자*130.3

Topic # 2
하늘*263.0+세상*249.1+모두*241.3+오늘*194.9+바람*161.5+노래*158.4+시간*155.2+위로*150.7+생각*137.3+사람*124.4+하루*121.1+순간*119.1+모습*117.2+가슴*117.1+거리*106.7

Topic # 3
사랑*1741.9+눈물*290.2+마음*286.1+가슴*284.5+세상*283.1+그녀*277.2+나나*258.3+사랑 사랑*252.9+나나 나나*192.3+사람*191.9+여자*161.0+당신*150.7+남자*139.0+때문*137.5+생각*133.8


In [22]:
# transform까지 수행하면, 문서별(row)로 토픽들(column)의 분포를 알려줌
doc_topics = lda.transform(ftr_vect)
print(doc_topics.shape)
print(doc_topics[:2])

(1028, 3)
[[0.4989286  0.01011064 0.49096076]
 [0.92475247 0.00858939 0.06665814]]


In [24]:
# 주어진 내장 텍스트데이터의 문서이름에는 카테고리가 labeling되어있음. 
# 따라서, 카테고리가 무엇인지 아는 상태이니까 어떤 문서들이 어떤 토픽들이 높은지 확인해보자.
# 그리고 그 토픽들이 각각 무엇을 내용으로 하는지 추측해보자.
# 주어진 데이터셋의 filename속성을 이용해서 카테고리값들 가져오기
def get_filename_list(year_df):
    filename_lst = []
    for file in year_df.song_name:
        filename_temp = file.split('/')[-2:]
        filename = '.'.join(filename_temp)
        filename_lst.append(filename)
    return filename_lst
 
filename_lst = get_filename_list(year_df)
# Dataframe형태로 만들어보기
topic_names = ['Topic #'+ str(i) for i in range(0,3)]
topic_df = pd.DataFrame(data=doc_topics, columns=topic_names,
                       index=filename_lst)
# print(topic_df.head(20))

In [25]:
topic_df[10:20]

,Topic #0,Topic #1,Topic #2
Missing You,0.022091,0.020750,0.957159
Abracadabra,0.013907,0.013842,0.972252
한 남자,0.008807,0.062196,0.928997
사랑은...향기를 남기고,0.008329,0.008453,0.983218
"너에게 난, 나에게 넌",0.007443,0.984889,0.007668
그런가봐요...,0.004951,0.004647,0.990402
잊지 말아요,0.659807,0.167328,0.172865
소원을 말해봐 (Genie),0.056457,0.219468,0.724075
그랬나봐,0.964106,0.014996,0.020898
Heartbreaker,0.107537,0.011825,0.880638


## Topic # 1
그대*1751.9+사랑*1129.0+사람*392.7+눈물*306.9+기억*235.9+그대 사랑*207.3+사랑 그대*205.8+그대 그대*170.3+생각*164.9+마음*148.2+모두*148.0+시간*147.2+가슴*137.4+바보*135.0+혼자*130.3

In [26]:
topic_df[topic_df['Topic #0']>=0.7].head(20)

,Topic #0,Topic #1,Topic #2
사랑합니다...,0.924752,0.008589,0.066658
소주 한 잔,0.886860,0.105071,0.008069
...안 되나요... (부제: 화양연화),0.987052,0.005927,0.007021
그랬나봐,0.964106,0.014996,0.020898
내 귀에 캔디 (feat. 택연 Of 2PM),0.918879,0.076065,0.005055
Monologue,0.984210,0.006944,0.008846
잊었니,0.889072,0.100849,0.010079
Sweet Dream,0.729190,0.263362,0.007449
긴 하루,0.789546,0.202407,0.008047
그댄 달라요,0.984470,0.008357,0.007173


## Topic # 2
Topic # 2
하늘*263.0+세상*249.1+모두*241.3+오늘*194.9+바람*161.5+노래*158.4+시간*155.2+위로*150.7+생각*137.3+사람*124.4+하루*121.1+순간*119.1+모습*117.2+가슴*117.1+거리*106.7

In [37]:
topic_df[topic_df['Topic #1']>=0.7].head(20)

,Topic #0,Topic #1,Topic #2
Never Ending Story,0.114825,0.871439,0.013736
"너에게 난, 나에게 넌",0.007443,0.984889,0.007668
챔피언,0.005653,0.988766,0.005581
미스터,0.010159,0.979239,0.010602
디스코왕,0.008222,0.769709,0.222070
낭만 고양이,0.010843,0.978342,0.010815
Muzik,0.010146,0.980040,0.009814
아틀란티스 소녀,0.253625,0.739388,0.006988
Ring Ding Dong,0.010900,0.977371,0.011729
Hot 뜨거,0.006672,0.837561,0.155767


## Topic # 3
사랑*1741.9+눈물*290.2+마음*286.1+가슴*284.5+세상*283.1+그녀*277.2+나나*258.3+사랑 사랑*252.9+나나 나나*192.3+사람*191.9+여자*161.0+당신*150.7+남자*139.0+때문*137.5+생각*133.8

In [28]:
topic_df[topic_df['Topic #2']>=0.7].head(20) # 

,Topic #0,Topic #1,Topic #2
어쩌면...,0.081702,0.004863,0.913436
With Me,0.007285,0.007652,0.985063
니가 참 좋아,0.015793,0.016316,0.967892
Missing You,0.022091,0.020750,0.957159
Abracadabra,0.013907,0.013842,0.972252
한 남자,0.008807,0.062196,0.928997
사랑은...향기를 남기고,0.008329,0.008453,0.983218
그런가봐요...,0.004951,0.004647,0.990402
소원을 말해봐 (Genie),0.056457,0.219468,0.724075
Heartbreaker,0.107537,0.011825,0.880638
